In [89]:
import pyrealsense2 as rs
import numpy as np
import cv2
import imutils

import time
import math

%matplotlib inline
from matplotlib import pyplot as plt

In [90]:
img = cv2.imread('./image/boxColor.png')
imgOrig = img.copy()

mask = np.zeros(img.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

In [91]:
font = cv2.FONT_HERSHEY_TRIPLEX
textString = ''
colorFPS = (255,255,0)
colorDist = (0, 255,255)
colorRect = (0, 255,255)

colorLine = (0, 255, 0)
sizeLine = 3
colorLineP = (0, 255, 255)
sizeLineP = 1

In [92]:
# grabcut global parameters
flagGrab = False
flagDispGrab = False
flagClickGrab = False

flagPaintYellow = False
flagPaintRed = False
flagRect = False

flagGrabFinish = False

grabCordLTX = -1
grabCordLTY = -1
grabCordRDX = -1
grabCordRDY = -1

grabImg = np.zeros((512,512,3), np.uint8)
grabImg2 = np.zeros((512,512,3), np.uint8)
grabImg3 = np.zeros((512,512,3), np.uint8)

grabPaintX = -1
grabPaintY = -1

rect = (-1, -1, -1, -1)
thickness = 5 

In [93]:
# mouse click function
def procMouseGrab(event,x,y,flags,param):
    global grabCordLTX, grabCordLTY, grabCordRDX, grabCordRDY
    global grabPaintX, grabPaintY
    global flagClickGrab, flagPaintYellow, flagPaintRed, flagRect
    global grabImg, grabImg2, grabImg3
    
    if event is cv2.EVENT_LBUTTONDOWN:
        if flagRect is True:
            if flags & cv2.EVENT_FLAG_CTRLKEY:
                grabPaintX = x
                grabPaintY = y
                flagPaintYellow = True
            elif flags & cv2.EVENT_FLAG_SHIFTKEY:
                grabPaintX = x
                grabPaintY = y
                flagPaintRed = True
        else:      
            if grabCordLTY != -1 and grabCordRDY != -1:
                grabImg = img.copy()
                grabImg3 = img.copy()

            grabCordLTX = x
            grabCordLTY = y
            flagClickGrab = True
            grabImg2 = grabImg.copy()
        
    elif event is cv2.EVENT_MOUSEMOVE:
        if flagClickGrab is True:
            grabCordRDX = x
            grabCordRDY = y
        elif flagPaintYellow is True or flagPaintRed is True:
            grabPaintX = x
            grabPaintY = y

    elif event is cv2.EVENT_LBUTTONUP:
        if flagPaintYellow is True:
            grabPaintX = x
            grabPaintY = y
            flagPaintYellow = False
        elif flagPaintRed is True:
            grabPaintX = x
            grabPaintY = y
            flagPaintRed = False
        else:    
            grabCordRDX = x
            grabCordRDY = y
            flagClickGrab = False

In [94]:
def procGrabUI():
    global flagGrab, flagDispGrab, flagClickGrab, flagPaintYellow, flagPaintRed, flagRect, flagGrabFinish
    global grabCordLTX, grabCordLTY, grabCordRDX, grabCordRDY
    global grabPaintX, grabPaintY
    global grabImg, grabImg2, grabImg3
    global mask, bgdModel, fgdModel, rect
    
    if flagGrab is True:
        grabImg = img.copy()
        grabImg3 = img.copy()
        cv2.namedWindow('GrabCutUI')
        cv2.setMouseCallback('GrabCutUI', procMouseGrab)
        cv2.imshow('GrabCutUI', grabImg)
        flagDispGrab= True
        flagGrab = False
    
    if flagDispGrab is True:
        
        if flagPaintYellow is True and flagRect is True:
            cv2.circle(grabImg, (grabPaintX,grabPaintY), thickness, (0, 255, 255), -1)
            cv2.circle(mask, (grabPaintX,grabPaintY), thickness, 1, -1)      
            cv2.imshow('GrabCutUI', grabImg)
            
            if flagGrabFinish is True:
                cv2.circle(grabImg3, (grabPaintX,grabPaintY), thickness, (0, 255, 255), -1)
                cv2.imshow('GRABCUT', grabImg3)
        
        if flagPaintRed is True and flagRect is True:
            cv2.circle(grabImg, (grabPaintX,grabPaintY), thickness, (0, 0, 255), -1)
            cv2.circle(mask, (grabPaintX,grabPaintY), thickness, 0, -1)
            cv2.imshow('GrabCutUI', grabImg)
            
            if flagGrabFinish is True:
                cv2.circle(grabImg3, (grabPaintX,grabPaintY), thickness, 0, -1) 
                cv2.imshow('GRABCUT', grabImg3)

        if flagClickGrab is True:
            grabImg2 = grabImg.copy()
            if grabCordLTX != -1 and grabCordRDX != -1:
                cv2.rectangle(grabImg2, (grabCordLTX,grabCordLTY), (grabCordRDX, grabCordRDY), colorLineP, sizeLineP)
            cv2.imshow('GrabCutUI', grabImg2)
        else:
            if grabCordLTX != -1 and grabCordRDX != -1:
                cv2.rectangle(grabImg, (grabCordLTX,grabCordLTY), (grabCordRDX, grabCordRDY), colorLine, sizeLine)
                
                rect = (grabCordLTX, grabCordLTY, grabCordRDX, grabCordRDY)
                cv2.grabCut(grabImg3, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)            
                flagRect = True

                grabCordLTX = -1
                grabCordRDX = -1

            cv2.imshow('GrabCutUI', grabImg)

In [95]:
def procGrabCut():
    global grabImg3
    global mask, bgdModel, fgdModel, rect
    global flagGrabFinish
    
    cv2.grabCut(grabImg3, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)   
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    grabImg3 = grabImg3 * mask2[:,:,np.newaxis]
    cv2.imshow('GRABCUT', grabImg3)
    flagGrabFinish = True

In [96]:
while True:
    cv2.imshow('Original', img)
    procGrabUI()
    
    
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('q') or getKey is ord('Q'):
        break
    elif getKey is ord('g') or getKey is ord('G'):
        flagGrab = True
    elif getKey is ord('h') or getKey is ord('H'):
        if flagRect is True:
            procGrabCut()
    elif getKey is ord('x') or getKey is ord('X'):
        cv2.destroyAllWindows()
        flagDispGrab = False
        flagGrabFinish = False
        cv2.namedWindow('GrabCutUI')
        cv2.setMouseCallback('GrabCutUI', procMouseGrab)

In [97]:
cv2.destroyAllWindows()